In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

with open("data/test.txt", "r", encoding="utf-8") as f:
    docs = [line.strip() for line in f if line.strip()]


model = SentenceTransformer("all-mpnet-base-v2")

embeddings = model.encode(docs, normalize_embeddings=True)

dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)


def ask(query, k=3):
    q_vec = model.encode([query], normalize_embeddings=True)
    D, I = index.search(np.array(q_vec), k)
    context = "\n".join([f"[{i+1}] {docs[j]}" for i, j in enumerate(I[0])])
    prompt = f"请根据以下资料回答问题：\n{context}\n\n问题：{query}"
    return prompt


ask("通过最近的国际新闻预测金价走势?")

/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bierxiensi/Desktop/flood-rag/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'请根据以下资料回答问题：\n[1] 王力宏回应是否想找新伴侣。\n[2] 第七条：公司是企业法人，有独立的法人财产。\n[3] 股东大会是公司的最高权力机构。\n\n问题：通过最近的国际新闻预测金价走势?'